In [ ]:
!pip install dash
import numpy as np
import plotly.graph_objects as go
from scipy.stats import beta, binom,gamma, norm
from dash import Dash, html, dcc, Input, Output
import math

In [ ]:

# Funções de probabilidade
def plot_beta_distribution(a, b,v="Priori"):
    # Gera valores de theta no intervalo (0,1), evitando extremos
    theta = np.linspace(0, 1, 1000)

    # Se for a Beta(1,1), a densidade é sempre 1
    if a == 1 and b == 1:
        beta_pdf = np.ones_like(theta)
    else:
        from scipy.stats import beta
        beta_pdf = beta.pdf(theta, a, b)

    # Cria o gráfico usando Plotly
    fig = go.Figure()

    # Adiciona a curva da densidade
    fig.add_trace(go.Scatter(
        x=theta,
        y=beta_pdf,
        mode='lines',
        name=f'Beta({a}, {b})',
        line=dict(color='royalblue', width=2)
    ))

    # Configurações do gráfico
    fig.update_layout(
        title=f'{v} Beta({a}, {b})',
        xaxis_title='Suporte do parâmetro',
        yaxis_title='Densidade',
        template='plotly_white',
        showlegend=True
    )
    return fig


def plot_gamma_distribution(a, b,v="Priori"):
    # Calcula a moda da distribuição Gamma (somente para a > 1)
    if a > 1:
        mode = (a - 1) / b
    else:
        mode = 0  # Não há moda definida para a <= 1
    # Ajusta o intervalo de x ao redor da moda, aumentando o intervalo
    x_min = max(0, mode - 3*np.sqrt(a/b**2))  # Aumenta o limite mínimo
    x_max = mode + 3*np.sqrt(a/b**2)          # Aumenta o limite máximo

    # Gera valores de x no intervalo ajustado
    x = np.linspace(x_min, x_max, 1000)

    # Calcula a densidade da Gamma(a, scale=1/b)
    gamma_pdf = gamma.pdf(x, a, scale=1/b)

    # Cria o gráfico usando Plotly
    fig = go.Figure()

    # Adiciona a curva da densidade
    fig.add_trace(go.Scatter(
        x=x,
        y=gamma_pdf,
        mode='lines',
        name=f'Gama({a}, {b})',
        line=dict(color='royalblue', width=2)
    ))

    # Personalizações do layout
    fig.update_layout(
        title=f'{v} Gama({a}, {b})',
        xaxis_title='Suporte do parâmetro',
        yaxis_title='Densidade',
        template='plotly_white',
        showlegend=True
    )

    # Exibe a figura
    return fig

def plot_normal_distribution(mu, sigma2,v="Priori"):
    # Desvio padrão é a raiz quadrada da variância
    sigma = np.sqrt(sigma2)

    # Gera valores de x ao redor da média (mu), cobrindo a maior parte da densidade
    x = np.linspace(mu - 4*sigma, mu + 4*sigma, 1000)

    # Calcula a função densidade de probabilidade (PDF) da Normal
    normal_pdf = norm.pdf(x, mu, sigma)

    # Cria o gráfico usando Plotly
    fig = go.Figure()

    # Adiciona a curva da densidade
    fig.add_trace(go.Scatter(
        x=x,
        y=normal_pdf,
        mode='lines',
        name=f'Normal({mu}, {sigma2})',
        line=dict(color='royalblue', width=2)
    ))

    # Configurações do gráfico
    fig.update_layout(
        title=f'{v} Normal({mu}, {sigma2})',
        xaxis_title='Suporte do parâmetro',
        yaxis_title='Densidade',
        template='plotly_white',
        showlegend=True
    )

    return fig


def verossimilhanca_binomial_aproximada(x,m,n):
  figura=plot_beta_distribution(round(n*x+1,3),round(n*(m-x)+1,3))
  figura.data[0].line.color = 'red'
  figura.update_layout(
      title="Verossimilhança reescalada da Binomial",
      yaxis_title="Verossimilhança"
  )
  return figura

def verossimilhanca_poisson_aproximada(x,n):
  figura=plot_gamma_distribution(round(n*x+1,3),n)
  figura.data[0].line.color = 'red'
  figura.update_layout(
      title="Verossimilhança reescalada da Poisson",
      yaxis_title="Verossimilhança",
  )
  return figura

def verossimilhanca_binomial_negativa_aproximada(x,r,n):
  figura=plot_beta_distribution(round(n*r+1,3),round(n*(x-r)+1,3))
  figura.data[0].line.color = 'red'
  figura.update_layout(
      title="Verossimilhança reescalada da Binomial negativa",
      yaxis_title="Verossimilhança"
  )
  return figura
def verossimilhanca_gama_aproximada(x,a,n):
  figura=plot_gamma_distribution(round(n*a+1,3),round(n*x,3))
  figura.data[0].line.color = 'red'
  figura.update_layout(
      title="Verossimilhança reescalada da Gama",
      yaxis_title="Verossimilhança"
  )
  return figura
def verossimilhanca_exponencial_aproximada(x,n):
  figura=plot_gamma_distribution(n+1,round(n*x,3))
  figura.data[0].line.color = 'red'
  figura.update_layout(
      title="Verossimilhança reescalada da Exponencial",
      yaxis_title="Verossimilhança"
  )
  return figura

def verossimilhanca_normal_aproximada(x,sigma2,n):
  figura=plot_normal_distribution(x,round(sigma2/n,3))
  figura.data[0].line.color = 'red'
  figura.update_layout(
      title="Verossimilhança reescalada da Normal",
      yaxis_title="Verossimilhança"
  )
  return figura

def verossimilhanca_bernoulli_aproximada(x,n):
  figura=plot_beta_distribution(round(n*x+1,3),round(n*(1-x)+1,3))
  figura.data[0].line.color = 'red'
  figura.update_layout(
      title="Verossimilhança reescalada da Bernoulli",
      yaxis_title="Verossimilhança"
  )
  return figura
def verossimilhanca_geometrica_aproximada(x,n):
  figura=plot_beta_distribution(n+1,round(n*(x-1)+1,3))
  figura.data[0].line.color = 'red'
  figura.update_layout(
      title="Verossimilhança reescalada da Geométrica",
      yaxis_title="Verossimilhança"
  )
  return figura

def beta_pdf(a,b):
    theta = np.linspace(0, 1, 1000)
    if a == 1 and b == 1:
        beta_pdf = np.ones_like(theta)
    else:
        from scipy.stats import beta
        beta_pdf = beta.pdf(theta, a, b)
    return beta_pdf

def beta_binomial(a, b,x,m,n):
    # Cria o gráfico usando Plotly
    fig = go.Figure()
    eixo_x=np.linspace(0,1,1000)
    y_priori=beta_pdf(a,b)
    y_verossimilhanca=beta_pdf(round(n*x+1,3),round(n*(m-x)+1,3))
    y_posteriori=beta_pdf(round(n*x+a,3),round(b+n*(m-x),3))
    # Adiciona a curva da densidade da priori
    fig.add_trace(go.Scatter(
        x=eixo_x,
        y=y_priori,
        mode='lines',
        name=f'Beta({a}, {b})',
        line=dict(color='royalblue', width=2)
    ))
    fig.add_trace(go.Scatter(
        x=eixo_x,
        y=y_verossimilhanca,
        mode='lines',
        name=f'Beta({round(n*x+1,3)}, {round(n*(m-x)+1,3)})',
        line=dict(color='red', width=2)
    ))
    fig.add_trace(go.Scatter(
        x=eixo_x,
        y=y_posteriori,
        mode='lines',
        name=f'Beta({round(n*x+a,3)}, {round(b+n*(m-x),3)})',
        line=dict(color='green', width=2)
    ))
    # Configurações do gráfico
    fig.update_layout(
        title=f'Distribuição Beta-Binomial',
        xaxis_title='Suporte do parâmetro',
        yaxis_title='Densidade',
        template='plotly_white'
    )
    return fig

def beta_bernoulli(a,b,x,n):
    # Cria o gráfico usando Plotly
    fig = go.Figure()
    eixo_x=np.linspace(0,1,1000)
    y_priori=beta_pdf(a,b)
    y_verossimilhanca=beta_pdf(round(n*x+1,3),round(n*(1-x)+1,3))
    y_posteriori=beta_pdf(round(a+n*x,3),round(b+n*(1-x),3))
    # Adiciona a curva da densidade da priori
    fig.add_trace(go.Scatter(
        x=eixo_x,
        y=y_priori,
        mode='lines',
        name=f'Beta({a}, {b})',
        line=dict(color='royalblue', width=2)
    ))
    fig.add_trace(go.Scatter(
        x=eixo_x,
        y=y_verossimilhanca,
        mode='lines',
        name=f'Beta({round(n*x+1,3)}, {round(n*(1-x)+1,3)})',
        line=dict(color='red', width=2)
    ))
    fig.add_trace(go.Scatter(
        x=eixo_x,
        y=y_posteriori,
        mode='lines',
        name=f'Beta({round(a+n*x,3)}, {round(b+n*(1-x),3)})',
        line=dict(color='green', width=2)
    ))
    # Configurações do gráfico
    fig.update_layout(
        title=f'Distribuição Beta-Bernoulli',
        xaxis_title='Suporte do parâmetro',
        yaxis_title='Densidade',
        template='plotly_white'
    )
    return fig

def beta_geometrica(a,b,x,n):
    # Cria o gráfico usando Plotly
    fig = go.Figure()
    eixo_x=np.linspace(0,1,1000)
    y_priori=beta_pdf(a,b)
    y_verossimilhanca=beta_pdf(n+1,round(n*(x-1)+1,3))
    y_posteriori=beta_pdf(a+n,round(b+n*(x-1),3))
    # Adiciona a curva da densidade da priori
    fig.add_trace(go.Scatter(
        x=eixo_x,
        y=y_priori,
        mode='lines',
        name=f'Beta({a}, {b})',
        line=dict(color='royalblue', width=2)
    ))
    fig.add_trace(go.Scatter(
        x=eixo_x,
        y=y_verossimilhanca,
        mode='lines',
        name=f'Beta({n+1}, {round(n*(x-1)+1,3)})',
        line=dict(color='red', width=2)
    ))
    fig.add_trace(go.Scatter(
        x=eixo_x,
        y=y_posteriori,
        mode='lines',
        name=f'Beta({a+n}, {round(b+n*(x-1),3)})',
        line=dict(color='green', width=2)
    ))
    # Configurações do gráfico
    fig.update_layout(
        title=f'Distribuição Beta-Geométrica',
        xaxis_title='Suporte do parâmetro',
        yaxis_title='Densidade',
        template='plotly_white'
    )
    return fig

def beta_binomial_negativa(a,b,x,r,n):
    # Cria o gráfico usando Plotly
    fig = go.Figure()
    eixo_x=np.linspace(0,1,1000)
    y_priori=beta_pdf(a,b)
    y_verossimilhanca=beta_pdf(round(n*r+1,3),round(n*(x-r)+1,3))
    y_posteriori=beta_pdf(round(a+n*r,3),round(b+n*(x-r),3))
    # Adiciona a curva da densidade da priori
    fig.add_trace(go.Scatter(
        x=eixo_x,
        y=y_priori,
        mode='lines',
        name=f'Beta({a}, {b})',
        line=dict(color='royalblue', width=2)
    ))
    fig.add_trace(go.Scatter(
        x=eixo_x,
        y=y_verossimilhanca,
        mode='lines',
        name=f'Beta({round(n*r+1,3)}, {round(n*(x-r)+1,3)})',
        line=dict(color='red', width=2)
    ))
    fig.add_trace(go.Scatter(
        x=eixo_x,
        y=y_posteriori,
        mode='lines',
        name=f'Beta({round(a+n*r,3)}, {round(b+n*(x-r),3)})',
        line=dict(color='green', width=2)
    ))
    # Configurações do gráfico
    fig.update_layout(
        title=f'Distribuição Beta-Binomial negativa',
        xaxis_title='Suporte do parâmetro',
        yaxis_title='Densidade',
        template='plotly_white'
    )
    return fig

def gama_pdf(a, b):
    if a > 1:
        mode = (a - 1) / b
    else:
        mode = 0
    x_min = max(0, mode - 3*np.sqrt(a/b**2))
    x_max = mode + 3*np.sqrt(a/b**2)
    return x_min,x_max

def gama_exponencial(a,b,x,n):
  x_min_priori,x_max_priori=gama_pdf(a,b)
  x_min_verossimilhanca,x_max_verossimilhanca=gama_pdf(n+1,n*x)
  x_min_posteriori,x_max_posteriori=gama_pdf(a+n,b+n*x)
  x_min=min(x_min_priori,x_min_verossimilhanca,x_min_posteriori)
  x_max=max(x_max_priori,x_max_verossimilhanca,x_max_posteriori)
  eixo_x=np.linspace(x_min,x_max,1000)
  y_priori=gamma.pdf(eixo_x,a,scale=1/b)
  y_verossimilhanca=gamma.pdf(eixo_x,n+1,scale=round(1/(n*x),3))
  y_posteriori=gamma.pdf(eixo_x,a+n,scale=round(1/(b+n*x),3))
  figura=go.Figure()
  figura.add_trace(go.Scatter(
        x=eixo_x,
        y=y_priori,
        mode='lines',
        name=f'Gama({a}, {b})',
        line=dict(color='royalblue', width=2)
    ))
  figura.add_trace(go.Scatter(
        x=eixo_x,
        y=y_verossimilhanca,
        mode='lines',
        name=f'Gama({n+1}, {round(n*x,3)})',
        line=dict(color='red', width=2)
    ))
  figura.add_trace(go.Scatter(
        x=eixo_x,
        y=y_posteriori,
        mode='lines',
        name=f'Gama({a+n}, {round(b+n*x,3)})',
        line=dict(color='green', width=2)
    ))
  figura.update_layout(
      title=f'Distribuição Gama-Exponencial',
      xaxis_title='Suporte do parâmetro',
      yaxis_title='Densidade',
      template='plotly_white',
      showlegend=True
    )
  return figura

def gama_poisson(a,b,x,n):
  x_min_priori,x_max_priori=gama_pdf(a,b)
  x_min_verossimilhanca,x_max_verossimilhanca=gama_pdf(n*x+1,n)
  x_min_posteriori,x_max_posteriori=gama_pdf(n*x+a,b+n)
  x_min=min(x_min_priori,x_min_verossimilhanca,x_min_posteriori)
  x_max=max(x_max_priori,x_max_verossimilhanca,x_max_posteriori)
  eixo_x=np.linspace(x_min,x_max,1000)
  y_priori=gamma.pdf(eixo_x,a,scale=1/b)
  y_verossimilhanca=gamma.pdf(eixo_x,round(n*x+1,3),scale=round(1/(n),3))
  y_posteriori=gamma.pdf(eixo_x,round(n*x+a,3),scale=round(1/(b+n),3))
  figura=go.Figure()
  figura.add_trace(go.Scatter(
        x=eixo_x,
        y=y_priori,
        mode='lines',
        name=f'Gama({a}, {b})',
        line=dict(color='royalblue', width=2)
    ))
  figura.add_trace(go.Scatter(
        x=eixo_x,
        y=y_verossimilhanca,
        mode='lines',
        name=f'Gama({round(n*x+1,3)}, {n})',
        line=dict(color='red', width=2)
    ))
  figura.add_trace(go.Scatter(
        x=eixo_x,
        y=y_posteriori,
        mode='lines',
        name=f'Gama({round(n*x+a,3)}, {b+n})',
        line=dict(color='green', width=2)
    ))
  figura.update_layout(
      title=f'Distribuição Gama-Poisson',
      xaxis_title='Suporte do parâmetro',
      yaxis_title='Densidade',
      template='plotly_white',
      showlegend=True
    )
  return figura

def gama_gama(a,b,x,conhecido,n):
  x_min_priori,x_max_priori=gama_pdf(a,b)
  x_min_verossimilhanca,x_max_verossimilhanca=gama_pdf(n*conhecido+1,n*x)
  x_min_posteriori,x_max_posteriori=gama_pdf(a+n*conhecido,b+n*x)
  x_min=min(x_min_priori,x_min_verossimilhanca,x_min_posteriori)
  x_max=max(x_max_priori,x_max_verossimilhanca,x_max_posteriori)
  eixo_x=np.linspace(x_min,x_max,1000)
  y_priori=gamma.pdf(eixo_x,a,scale=1/b)
  y_verossimilhanca=gamma.pdf(eixo_x,round(n*conhecido+1,3),scale=round(1/(n*x),3))
  y_posteriori=gamma.pdf(eixo_x,round(a+n*conhecido,3),scale=round(1/(b+n*x),3))
  figura=go.Figure()
  figura.add_trace(go.Scatter(
        x=eixo_x,
        y=y_priori,
        mode='lines',
        name=f'Gama({a}, {b})',
        line=dict(color='royalblue', width=2)
    ))
  figura.add_trace(go.Scatter(
        x=eixo_x,
        y=y_verossimilhanca,
        mode='lines',
        name=f'Gama({round(n*conhecido+1,3)}, {round(n*x,3)})',
        line=dict(color='red', width=2)
    ))
  figura.add_trace(go.Scatter(
        x=eixo_x,
        y=y_posteriori,
        mode='lines',
        name=f'Gama({round(a+n*conhecido,3)}, {round(b+n*x,3)})',
        line=dict(color='green', width=2)
    ))
    # Personalizações do layout
  figura.update_layout(
      title=f'Distribuição Gama-Gama',
      xaxis_title='Suporte do parâmetro',
      yaxis_title='Densidade',
      template='plotly_white',
      showlegend=True
    )

    # Exibe a figura
  return figura

def normal_pdf(mu,sigma2):
    sigma = np.sqrt(sigma2)
    return mu - 4*sigma,  mu + 4*sigma
def normal_normal(mu,sigma2,x,conhecido,n):
  x_min_priori,x_max_priori=normal_pdf(mu,sigma2)
  x_min_verossimilhanca,x_max_verossimilhanca=normal_pdf(x,conhecido/n)
  x_min_posteriori,x_max_posteriori=normal_pdf((n*sigma2*x+conhecido*mu)/(n*sigma2+conhecido),sigma2*conhecido/(n*sigma2+conhecido))
  x_min=min(x_min_priori,x_min_verossimilhanca,x_min_posteriori)
  x_max=max(x_max_priori,x_max_verossimilhanca,x_max_posteriori)
  eixo_x=np.linspace(x_min,x_max,1000)
  y_priori=norm.pdf(eixo_x,mu,np.sqrt(sigma2))
  y_verossimilhanca=norm.pdf(eixo_x,x,round(np.sqrt(conhecido/n),3))
  y_posteriori=norm.pdf(eixo_x,round((n*sigma2*x+conhecido*mu)/(n*sigma2+conhecido),3),round(np.sqrt(sigma2*conhecido/(n*sigma2+conhecido)),3))
  figura = go.Figure()

  figura.add_trace(go.Scatter(
        x=eixo_x,
        y=y_priori,
        mode='lines',
        name=f'Normal({mu}, {sigma2})',
        line=dict(color='royalblue', width=2)
  ))
  figura.add_trace(go.Scatter(
        x=eixo_x,
        y=y_verossimilhanca,
        mode='lines',
        name=f'Normal({x}, {round(conhecido/n,3)})',
        line=dict(color='red', width=2)
  ))
  figura.add_trace(go.Scatter(
        x=eixo_x,
        y=y_posteriori,
        mode='lines',
        name=f'Normal({round((n*sigma2*x+conhecido*mu)/(n*sigma2+conhecido),3)}, {round(sigma2*conhecido/(n*sigma2+conhecido),3)})',
        line=dict(color='green', width=2)
  ))
  figura.update_layout(
        title=f'Distribuição Normal-Normal',
        xaxis_title='Suporte do parâmetro',
        yaxis_title='Densidade',
        template='plotly_white'
  )

  return figura

def posteriori_beta(a,b,v="Posteriori"):
  figura=plot_beta_distribution(a,b,v)
  figura.data[0].line.color = 'green'
  return figura

def posteriori_gama(a,b,v="Posteriori"):
  figura=plot_gamma_distribution(a,b,v)
  figura.data[0].line.color = 'green'
  return figura

def posteriori_normal(a,b,v="Posteriori"):
  figura=plot_normal_distribution(a,b,v)
  figura.data[0].line.color = 'green'
  return figura

def normal_gama_pdf(x, tau, mu, lambda_, alpha, beta):
  densidade=(beta**alpha)*np.sqrt(lambda_)*tau**(alpha-0.5)*np.exp(-beta*tau)*np.exp(-lambda_*tau*(x-mu)**2/2)/(math.gamma(alpha)*np.sqrt(2*np.pi))
  return densidade

colorscale_blue = [[0.0, "lightblue"],[0.5, "blue"],[1.0, "darkblue"]]
colorscale_red = [[0.0, "lightcoral"],[0.5, "red"],[1.0, "darkred"]]
colorscale_green = [[0.0, "lightgreen"],[0.5, "green"],[1.0, "darkgreen"]]

def plot_normal_gama_distribution(mu,lambda_,alpha,beta,cor=colorscale_blue,v="Priori"):
  mode_tau=(alpha-0.5)/beta
  desvio_x=np.sqrt(beta/(lambda_*(alpha-1)))
  desvio_tau=np.sqrt(alpha/beta**2)
  x_vals=np.linspace(mu-3*desvio_x,mu+3*desvio_x,100)
  tau_vals=np.linspace(max(mode_tau-3*desvio_tau,0),mode_tau+3*desvio_tau,100)
  X, T = np.meshgrid(x_vals, tau_vals)

    # Calculando a densidade para cada ponto (X, T)
  Z = normal_gama_pdf(X, T, mu, lambda_, alpha, beta)

    # Criando o gráfico 3D
  fig = go.Figure(data=[go.Surface(z=Z, x=X, y=T, colorscale=cor)])

    # Ajustando rótulos e layout
  fig.update_layout(title=f"{v} Normal-Gama({round(mu,3)}, {round(lambda_,3)}, {round(alpha,3)}, {round(beta,3)})",
                    scene=dict(
                    xaxis_title="x",
                    yaxis_title="tau",
                    zaxis_title="Densidade"
                      ))

  return fig

def verossimilhanca_normal_gama_aproximada(x,s,n):
  figura=plot_normal_gama_distribution(x,n,(n+1)/2,n*s/2,colorscale_red)
  figura.update_layout(title="Verossimilhança aproximada da Normal",
                    scene=dict(
                    xaxis_title="x",
                    yaxis_title="tau",
                    zaxis_title="Verossimilhança"
                      ))
  return figura

def posteriori_Normal_Gama(mu,lambda_,alpha,beta,x,s,n,v="Posteriori"):
  figura=plot_normal_gama_distribution((lambda_*mu+n*x)/(lambda_+n),lambda_+n,alpha+n/2,beta+(n*s+(lambda_*n*(x-mu)**2)/(lambda_+n))/2,colorscale_green,v)
  return figura

def xvals_tauvals(mu,lambda_,alpha,beta):
  mode_tau=(alpha-0.5)/beta
  desvio_x=np.sqrt(beta/(lambda_*(alpha-1)))
  desvio_tau=np.sqrt(alpha/beta**2)
  x_vals=np.linspace(mu-3*desvio_x,mu+3*desvio_x,100)
  tau_vals=np.linspace(max(mode_tau-3*desvio_tau,0),mode_tau+3*desvio_tau,100)
  return x_vals,tau_vals
def Normal_Gama_final(mu,lambda_,alpha,beta,x,s,n):
  x_priori,tau_priori=xvals_tauvals(mu,lambda_,alpha,beta)
  x_verossimilhanca,tau_verossimilhanca=xvals_tauvals(x,n,(n+1)/2,n*s/2)
  x_posteriori,tau_posteriori=xvals_tauvals((lambda_*mu+n*x)/(lambda_+n),lambda_+n,alpha+n/2,beta+(n*s+(lambda_*n*(x-mu)**2)/(lambda_+n))/2)
  x_final=np.linspace(min(x_priori[0],x_verossimilhanca[0],x_posteriori[0]),max(x_priori[-1],x_verossimilhanca[-1],x_posteriori[-1]),100)
  tau_final=np.linspace(min(tau_priori[0],tau_verossimilhanca[0],tau_posteriori[0]),max(tau_priori[-1],tau_verossimilhanca[-1],tau_posteriori[-1]),100)
  X, T = np.meshgrid(x_final, tau_final)
  Z_priori = normal_gama_pdf(X, T, mu, lambda_, alpha, beta)
  Z_verossimilhanca = normal_gama_pdf(X, T, x, n, (n+1)/2, n*s/2)
  Z_posteriori = normal_gama_pdf(X, T, (lambda_*mu+n*x)/(lambda_+n), lambda_+n, alpha+n/2, (beta+(n*s+(lambda_*n*(x-mu)**2)/(lambda_+n))/2))
  fig = go.Figure(go.Surface(z=Z_priori, x=X, y=T, colorscale=colorscale_blue,name="Priori",showscale=False))
  fig.add_trace(go.Surface(z=Z_verossimilhanca, x=X, y=T, colorscale=colorscale_red,name="Verossimilhança",showscale=False))
  fig.add_trace(go.Surface(z=Z_posteriori, x=X, y=T, colorscale=colorscale_green,name="Posteriori",showscale=False))
  fig.add_trace(go.Scatter3d(x=[None], y=[None], z=[None], mode='markers', marker=dict(color='blue'), name="Priori"))
  fig.add_trace(go.Scatter3d(x=[None], y=[None], z=[None], mode='markers', marker=dict(color='red'), name="Verossimilhança"))
  fig.add_trace(go.Scatter3d(x=[None], y=[None], z=[None], mode='markers', marker=dict(color='green'), name="Posteriori"))
  fig.update_layout(title="Distribuição Normal-Gama",
                    showlegend=True,
                    scene=dict(
                    xaxis_title="x",
                    yaxis_title="tau",
                    zaxis_title="Densidade"
                      ))
  return fig


In [ ]:

app = Dash(__name__)

lista_verossimilhancas=list(["Bernoulli","Binomial","Geométrica","Binomial negativa","Poisson","Exponencial","Gama","Normal"])
lista_prioris=list(["Beta","Gama","Normal","Normal-Gama"])

app.layout = html.Div(children=[
    html.H1(children='Trabalho de Bayesiana sobre distribuições conjugadas - Grupo PET Estatística'),
    html.H4('Selecione a distribuição da priori'),
    dcc.Dropdown(lista_prioris,value="Beta",id="prioris"),
    html.H4('Selecione o modelo estatístico'),
    dcc.Dropdown(value="Bernoulli",id="verossimilhancas"),
    html.H4('Digite os parâmetros da priori'),

    # Campo para digitar o primeiro parâmetro da priori
    html.Div([
        html.Div([
            html.Label(id="texto_priori_1"),
            dcc.Input(id='input-a', type='number',value=2),
        ],style={'marginBottom': '10px'}),


        # Campo para digitar o segundo parâmetro da priori
        html.Div([
            html.Label(id="texto_priori_2"),
            dcc.Input(id='input-b', type='number',min=0.001,value=4),
        ],style={'marginBottom': '10px'}),

        # Campo para digitar o terceiro parâmetro da priori
        html.Div([
            html.Label(id="texto_priori_3"),
            dcc.Input(id='input-c', type='number',min=1.001,value=3),
        ],style={'marginBottom': '10px'}),


        # Campo para digitar o quarto parâmetro da priori
        html.Div([
            html.Label(id="texto_priori_4"),
            dcc.Input(id='input-d', type='number',min=0.001,value=2),
        ],style={'marginBottom': '10px'}),
    ],style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'flex-start'}),

    html.Br(),

    dcc.Graph(figure=plot_beta_distribution(2,4),
        id='densidade_priori',
    ),
    html.H4('Digite as informações da verossimilhança'),

    # Campo para digitar o primeiro parâmetro da verossimilhança
    html.Div([
        html.Div([
            html.Label(id="texto_verossimilhanca_1"),
            dcc.Input(id='input-m', type='number',min=1,step=1,value=5),
        ],style={'marginBottom': '10px'}),


        # Campo para digitar os valores observados
        html.Div([
            html.Label("Digite a média amostral:"),
            dcc.Input(id='input-x', type='number',value=1),
            dcc.Input(id="input-x-bernoulli",type="number",step=0.01,min=0,max=1,value=0.5),
        ],style={'marginBottom': '10px'}),



        # Campo para digitar o tamanho amostral
        html.Div([
            html.Label(children="Digite o tamanho amostral:",id='texto_posteriori_2'),
            dcc.Input(id='input-tamanho', type='number',min=1,step=1,value=2),
        ],style={'marginBottom': '10px'}),


        # Campo para digitar o parâmetro conhecido
        html.Div([
            html.Label(id="texto_param_conhecido"),
            dcc.Input(id='input-conhecido', type='number',value=1,min=0.001),
        ],style={'marginBottom': '10px'}),
    ],style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'flex-start'}),



    # Gráfico da verossimilhança
    html.Br(),
    html.Div(
        dcc.Graph(figure=verossimilhanca_bernoulli_aproximada(0.5,2),
        id='densidade_verossimilhanca',),
        id='aparencia_verossimilhanca'),
    # Gráfico da posteriori
    html.Br(),
    dcc.Graph(figure=plot_beta_distribution(3,5),id="densidade_posteriori"),

    # Gráfico final
    dcc.Graph(figure=beta_bernoulli(2,4,0.5,2),id="grafico_conjunto"),

    # Texto em latex
    html.Button("Aplicar valores", id="botao", n_clicks=0),
    dcc.Markdown(id="texto_botao",mathjax=True)
],    style={
        "backgroundColor": "#f0f0f0",  # Cor de fundo
        "color": "#333",              # Cor do texto
        "padding": "20px",            # Espaçamento interno
        "fontFamily": "Arial"         # Fonte
    })



# Atualiza as opções de seleção de verossimilhança
@app.callback(
    Output("verossimilhancas","options"),
    Input("prioris","value")
)
def update_dropdown(prioris):
    if prioris=="Beta":
        return list(["Bernoulli","Geométrica","Binomial negativa"])
    elif prioris=="Gama":
        return list(["Exponencial","Poisson","Gama (b desconhecido)"])
    elif prioris=="Normal-Gama":
        return list(["Normal (média e precisão desconhecidas)"])
    else:
        return list(["Normal (média desconhecida)"])

# Atualiza o texto do campo de digitação do primeiro parâmetro da priori
@app.callback(
    [Output("texto_priori_1","children"),
     Output("input-a","min")],
    Input("prioris","value")
)
def update_dropdown(prioris):
    if prioris=="Beta" or prioris=="Gama":
        return "Digite o valor de a:",0.001
    elif prioris=="Normal-Gama":
        return "Digite o valor de μ da priori:",None
    else:
        return "Digite a média da priori:",None

# Atualiza o texto do campo de digitação do segundo parâmetro da priori
@app.callback(
    Output("texto_priori_2","children"),
    Input("prioris","value")
)
def update_dropdown(prioris):
    if prioris=="Beta" or prioris=="Gama":
        return "Digite o valor de b:"
    elif prioris=="Normal-Gama":
        return "Digite o valor de λ da priori:"
    else:
        return "Digite a variância da priori:"

# Atualiza o texto do campo de digitação do terceiro parâmetro da priori
@app.callback(
    [Output("texto_priori_3","children"),
     Output("input-c","style")],
    Input("prioris","value")
)
def update_dropdown(prioris):
    if prioris=="Normal-Gama":
      return "Digite o valor de a:",{}
    else:
      return "",{"display":"none"}

# Atualiza o texto do campo de digitação do quarto parâmetro da priori
@app.callback(
    [Output("texto_priori_4","children"),
     Output("input-d","style")],
    Input("prioris","value")
)
def update_dropdown(prioris):
    if prioris=="Normal-Gama":
      return "Digite o valor de b:",{}
    else:
      return "",{"display":"none"}

# Atualiza o gráfico da priori
@app.callback(
    Output('densidade_priori', 'figure'),
    [Input('input-a', 'value'),
     Input('input-b', 'value'),
     Input('input-c', 'value'),
     Input('input-d', 'value'),
     Input("prioris","value")]
)
def update_output(a, b,c,d,prioris):
  if(prioris=="Gama"):
    return plot_gamma_distribution(a,b)
  elif (prioris=="Beta"):
    return plot_beta_distribution(a,b)
  elif(prioris=="Normal-Gama"):
    return plot_normal_gama_distribution(a,b,c,d)
  else:
    return plot_normal_distribution(a,b)

# Atualiza o texto do campo de digitação do primeiro parâmetro da verossimilhança
@app.callback(
    [Output("texto_verossimilhanca_1","children"),
     Output("input-m","style")],
    Input("verossimilhancas","value")
)
def update_dropdown(verossimilhancas):
    if verossimilhancas=="Binomial":
        return "Digite o valor de m:",{}
    elif verossimilhancas=="Binomial negativa":
        return "Digite o valor de r:",{}
    else:
      return "",{"display":"none"}

# Atualiza o valor mínimo do tamanho amostral
@app.callback(
    Output("input-tamanho","min"),
    Input("verossimilhancas","value")
)
def update_tamanho(verossimilhancas):
    if verossimilhancas=="Normal (média e precisão desconhecidas)":
      return 2
    else:
      return 1


# Atualiza o texto do campo de digitação dos valores observados
@app.callback(
    [Output("input-x","min"),
     Output("input-x","style"),
     Output("input-x","max")],
    [Input("verossimilhancas","value"),
     Input("input-m","value")]
)
def update_dropdown(verossimilhancas,m):
    if verossimilhancas=="Bernoulli":
        return None,{"display":"none"},None
    elif verossimilhancas=="Binomial":
        return 0,{},m
    elif verossimilhancas=="Geométrica":
        return 1,{},None
    elif verossimilhancas=="Binomial negativa":
        return m,{},None
    elif verossimilhancas=="Poisson":
        return 0,{},None
    elif verossimilhancas=="Exponencial":
        return 0,{},None
    elif verossimilhancas=="Gama (b desconhecido)":
        return 0,{},None
    else:
        return None,{},None

@app.callback(
    Output("input-x-bernoulli","style"),
    Input("verossimilhancas","value")
)
def update_dropdown(verossimilhancas):
    if verossimilhancas=="Bernoulli":
        return {}
    else:
        return {"display":"none"}

# Atualiza o texto do campo de digitação do parâmetro conhecido
@app.callback(
    [Output("texto_param_conhecido","children"),
     Output("input-conhecido","style")],
    Input("verossimilhancas","value")
)
def update_dropdown(verossimilhancas):
    if verossimilhancas=="Normal (média desconhecida)":
        return "Digite a variância populacional:",{}
    elif verossimilhancas=="Gama (b desconhecido)":
        return "Digite o valor de a conhecido:",{}
    elif verossimilhancas=="Normal (média e precisão desconhecidas)":
        return "Digite a variância amostral:",{}
    else:
        return "",{"display":"none"}

# Atualiza o gráfico da verossimilhança
@app.callback(
    [Output('densidade_verossimilhanca', 'figure'),
     Output('aparencia_verossimilhanca', 'style')],
    [Input('input-m', 'value'),
     Input('input-x', 'value'),
     Input('input-x-bernoulli','value'),
     Input('input-tamanho','value'),
     Input('input-conhecido','value'),
     Input('verossimilhancas','value')]
)
def update_output(m, x,x_bernoulli, n,conhecido,verossimilhancas):
    if verossimilhancas=="Geométrica":
      return verossimilhanca_geometrica_aproximada(x,n),{'display':'block'}
    elif verossimilhancas=="Bernoulli":
      return verossimilhanca_bernoulli_aproximada(x_bernoulli,n),{'display':'block'}
    elif verossimilhancas=="Normal (média desconhecida)":
      return verossimilhanca_normal_aproximada(x,conhecido,n),{'display':'block'}
    elif verossimilhancas=="Exponencial":
      return verossimilhanca_exponencial_aproximada(x,n),{'display':'block'}
    elif verossimilhancas=="Binomial":
      return verossimilhanca_binomial_aproximada(x,m,n),{'display':'block'}
    elif verossimilhancas=="Poisson":
      return verossimilhanca_poisson_aproximada(x,n),{'display':'block'}
    elif verossimilhancas=="Binomial negativa":
      return verossimilhanca_binomial_negativa_aproximada(x,conhecido,n),{'display':'block'}
    elif verossimilhancas=="Gama (b desconhecido)":
      return verossimilhanca_gama_aproximada(x,conhecido,n),{'display':'block'}
    elif verossimilhancas=="Normal (média e precisão desconhecidas)":
      return verossimilhanca_normal_gama_aproximada(x,conhecido,n),{'display':'block'}
    else:
      return go.Figure(),{'display':'none'}

# Atualiza o gráfico da posteriori
@app.callback(
    Output('densidade_posteriori', 'figure'),
    [Input('input-a', 'value'),
     Input('input-b', 'value'),
     Input('input-c', 'value'),
     Input('input-d', 'value'),
     Input('input-m', 'value'),
     Input('input-x', 'value'),
     Input('input-x-bernoulli','value'),
     Input('input-tamanho', 'value'),
     Input('input-conhecido', 'value'),
     Input("prioris","value"),
     Input("verossimilhancas","value")]
)
def update_output(a,b,c,d,m,x,x_bernoulli,n,conhecido,prioris,verossimilhancas):
  if verossimilhancas=="Binomial":
    return posteriori_beta(round(n*x+a,3),round(b+n*(m-x),3))
  elif verossimilhancas=="Bernoulli":
    return posteriori_beta(round(a+n*x_bernoulli,3),round(b+n*(1-x_bernoulli),3))
  elif verossimilhancas=="Poisson":
    return posteriori_gama(round(n*x+a,3),b+n)
  elif verossimilhancas=="Exponencial":
    return posteriori_gama(a+n,round(b+n*x,3))
  elif verossimilhancas=="Gama (b desconhecido)":
    return posteriori_gama(round(a+n*conhecido,3),round(b+n*x,3))
  elif verossimilhancas=="Geométrica":
    return posteriori_beta(a+n,round(b+n*(x-1),3))
  elif verossimilhancas=="Binomial negativa":
    return posteriori_beta(round(a+n*conhecido,3),round(b+n*(x-conhecido),3))
  elif verossimilhancas=="Normal (média e precisão desconhecidas)":
    return posteriori_Normal_Gama(a,b,c,d,x,conhecido,n)
  else:
    return posteriori_normal(round((n*b*x+a*conhecido)/(n*b+conhecido),3),round(conhecido*b/(n*b+conhecido),3))

# Atualiza o gráfico final
@app.callback(
    Output('grafico_conjunto', 'figure'),
    [Input('input-a', 'value'),
     Input('input-b', 'value'),
     Input('input-c', 'value'),
     Input('input-d', 'value'),
     Input('input-m', 'value'),
     Input('input-x', 'value'),
     Input('input-x-bernoulli','value'),
     Input('input-tamanho', 'value'),
     Input('input-conhecido', 'value'),
     Input("prioris","value"),
     Input("verossimilhancas","value")]
)
def update_output(a,b,c,d,m,x,x_bernoulli,n,conhecido,prioris,verossimilhancas):
  if verossimilhancas=="Bernoulli":
    return beta_bernoulli(a,b,x_bernoulli,n)
  elif verossimilhancas=="Binomial":
    return beta_binomial(a,b,x,m,n)
  elif verossimilhancas=="Geométrica":
    return beta_geometrica(a,b,x,n)
  elif verossimilhancas=="Binomial negativa":
    return beta_binomial_negativa(a,b,x,conhecido,n)
  elif verossimilhancas=="Exponencial":
    return gama_exponencial(a,b,x,n)
  elif verossimilhancas=="Poisson":
    return gama_poisson(a,b,x,n)
  elif verossimilhancas=="Gama (b desconhecido)":
    return gama_gama(a,b,x,conhecido,n)
  elif verossimilhancas=="Normal (média e precisão desconhecidas)":
    return Normal_Gama_final(a,b,c,d,x,conhecido,n)
  else:
    return normal_normal(a,b,x,conhecido,n)

# Atualiza o texto
@app.callback(
    Output("texto_botao","children"),
    [Input("botao","n_clicks"),
     Input("verossimilhancas","value"),
     Input("input-a","value"),
     Input('input-b', 'value'),
     Input('input-c', 'value'),
     Input('input-d', 'value'),
     Input('input-m', 'value'),
     Input('input-x', 'value'),
     Input('input-x-bernoulli','value'),
     Input('input-tamanho', 'value'),
     Input('input-conhecido', 'value')]
)
def update_output(n_clicks,verossimilhancas,a,b,c,d,m,x,x_bernoulli,n,conhecido):
    if n_clicks % 2 == 0:
      if verossimilhancas=="Bernoulli":
        return r'''
### Fórmulas matemáticas:

#### Priori:

$p \sim Beta(a,b)$

$f(p)=\frac{\Gamma(a+b) p^{a-1}(1-p)^{b-1}}{\Gamma(a)\Gamma(b)}\mathbb{I}_{(0,1)}(p)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $Bernoulli(p)$

#### Verossimilhança: $L(p|\mathbf{X})=p^{n\bar{x}}(1-p)^{(n-1)\bar{x}}$

#### Posteriori:

$p|\mathbf{X}\sim Beta(a+n\bar{x}, b+n(1-\bar{x}))$
    '''
      elif verossimilhancas=="Binomial":
        return r'''
### Fórmulas matemáticas:

#### Priori:

$p \sim Beta(a,b)$

$f(p)=\frac{\Gamma(a+b) p^{a-1}(1-p)^{b-1}}{\Gamma(a)\Gamma(b)}\mathbb{I}_{(0,1)}(p)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $Binomial(m, p)$

#### Verossimilhança: $L(p|\mathbf{X})=\left(\prod_{i=1}^{n}  \binom{m}{x_i} \right) p^{n\bar{x}} (1 - p)^{n(m-\bar{x})}$

#### Posteriori:

$p|\mathbf{X}\sim Beta(n\bar{x} + a, b+n(m - \bar{x}))$
        '''
      elif verossimilhancas=="Geométrica":
        return r'''
### Fórmulas matemáticas:

#### Priori:

$p \sim Beta(a,b)$

$f(p)=\frac{\Gamma(a+b) p^{a-1}(1-p)^{b-1}}{\Gamma(a)\Gamma(b)}\mathbb{I}_{(0,1)}(p)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $\text{Geométrica}(p)$

#### Verossimilhança: $L(p|\mathbf{X})=p^n(1-p)^{n(\bar{x}-1)}$

#### Posteriori:

$p|\mathbf{X}\sim Beta(a + n, b+n(\bar{x} - 1))$
'''
      elif verossimilhancas=="Binomial negativa":
        return r'''
### Fórmulas matemáticas:

#### Priori:

$p \sim Beta(a,b)$

$f(p)=\frac{\Gamma(a+b) p^{a-1}(1-p)^{b-1}}{\Gamma(a)\Gamma(b)}\mathbb{I}_{(0,1)}(p)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $Binomial Negativa(r, p)$

#### Verossimilhança: $L(p|\mathbf{X})=\left( \prod_{i=1}^n  \binom{x_i-1}{r-1} \right) p^{nr}(1-p)^{n(\bar{x}-r)}$

#### Posteriori:

$p|\mathbf{X}\sim Beta(a + nr, b+n(\bar{x} - r))$
'''
      elif verossimilhancas=="Exponencial":
        return r'''
### Fórmulas matemáticas:

#### Priori:

$\lambda \sim Gama(a,b)$

$f(\lambda)=\frac{b^a \lambda^{a - 1} e^{-b \lambda}}{\Gamma(a)} \mathbb{I}_{(0, \infty)}(\lambda)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $Exponencial(\lambda)$

#### Verossimilhança: $L(\lambda|\mathbf{X})=\lambda^ne^{-\lambda n\bar{x}}$

#### Posteriori:

$\lambda|\mathbf{X}\sim Gama(a+n, b+n\bar{x})$
'''
      elif verossimilhancas=="Poisson":
        return r'''
### Fórmulas matemáticas:

#### Priori:

$\lambda \sim Gama(a,b)$

$f(\lambda)=\frac{b^a \lambda^{a - 1} e^{-b \lambda}}{\Gamma(a)} \mathbb{I}_{(0, \infty)}(\lambda)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $Poisson(\lambda)$

#### Verossimilhança: $L(\lambda|\mathbf{X})= \frac{e^{-\lambda}\lambda^{n\bar{x}}}{\prod_{i=1}^n x_i!}$

#### Posteriori:

$\lambda|\mathbf{X}\sim Gama(n\bar{x}+a, b+n)$
'''
      elif verossimilhancas=="Gama (b desconhecido)":
        return r'''
### Fórmulas matemáticas:

#### Priori:

$b \sim Gama(a_0,b_0)$

$f(b)=\frac{b_0^{a_0} b^{a_0 - 1} e^{-b_0 b}}{\Gamma(a_0)} \mathbb{I}_{(0, \infty)}(b)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $Gama(a,b)$

#### Verossimilhança: $L(b|\mathbf{X})=\frac{\left( \prod_{i=1}^n x_i \right)^{a-1} b^{na}e^{-nb\bar{x}}}{\Gamma(a)}$

#### Posteriori:

$p|\mathbf{X}\sim Gama(a_0 + na, b_0 + n \bar{x})$
'''
      elif verossimilhancas=="Normal (média desconhecida)":
        return r'''
### Fórmulas matemáticas:

#### Priori:

$\mu \sim Normal(\mu_0,\sigma^2_0)$

$f(\mu) = \frac{1}{\sqrt{2\pi \sigma^2_0}} \exp\left( -\frac{(\mu - \mu_0)^2}{2\sigma^2_0} \right) \mathbb{I}_{(-\infty, \infty)}(\mu)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $Normal \left(\mu,\sigma^2 \right)$

#### Verossimilhança: $L(\mu|\mathbf{X}) = \left(\frac{1}{\sqrt{2\pi \sigma^2}}\right)^n \exp\left(-\frac{1}{2\sigma^2}\left(n\mu^2-2\mu n\bar{x} + \sum_{i=1}^{n} x_i^2\right)\right)$

#### Posteriori:

$\mu|\mathbf{X}\sim Normal\left( \frac{n \sigma^2_0 \bar{x} + \sigma^2 \mu_0}{n \sigma^2_0 + \sigma^2} \,,\, \frac{\sigma^2 \sigma^2_0}{n \sigma^2_0 + \sigma^2}\right)$
'''
      else:
        return r'''
### Fórmulas matemáticas:

#### Priori:

$(\mu,\tau) \sim Normal-Gama(\mu_0,\lambda,a,b)$

$f(\mu,\tau)=\frac{b^a \sqrt{\lambda}\tau^{a-0.5}e^{-b\tau}} {\Gamma{(a)}\sqrt{2\pi}} exp\left( \frac{\lambda \tau \left( \mu-\mu_0 \right)^2}{2} \right)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $Normal(\mu,\tau^{-1})$

#### Verossimilhança: $L(\mu,\tau|\mathbf{X})=\frac{1}{(\sqrt{2\pi})^n}\tau^{\frac{n}{2}}exp(\frac{-ns^2\tau}{2})exp \left( \frac{-n\tau \left( \mu-\bar{x} \right)^2}{2} \right)$

#### Posteriori:

$(\mu,\tau)|\mathbf{X} \sim Normal-Gama\left( \frac{\lambda\mu_0 + n\bar{x}}{\lambda+n}, \lambda+n, a+\frac{n}{2}, b+\frac{1}{2}\left( ns^2 + \frac{\lambda n \left( \bar{x} - \mu_0 \right)^2}{\lambda+n} \right) \right)$
'''
    else:
      if verossimilhancas=="Bernoulli":
        return fr'''
### Fórmulas matemáticas:

#### Priori:

$p \sim Beta({a},{b})$

$f(p)=\frac{{\Gamma({a+b}) p^{a-1}(1-p)^{b-1}}}{{\Gamma({a})\Gamma({b})}}\mathbb{{I}}_{{(0,1)}}(p)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $Bernoulli(p)$

#### Verossimilhança: $L(p|\mathbf{{X}})=p^{{{n*x_bernoulli}}}(1-p)^{{{n*(1-x_bernoulli)}}}$

#### Posteriori:

$p|\mathbf{{X}}\sim Beta({a+n*x_bernoulli}, {b+n*(1-x_bernoulli)})$
'''
      elif verossimilhancas=="Binomial":
        return fr'''
### Fórmulas matemáticas:

#### Priori:

$p \sim Beta({a},{b})$

$f(p)=\frac{{\Gamma({a+b}) p^{a-1}(1-p)^{b-1}}}{{\Gamma({a})\Gamma({b})}}\mathbb{{I}}_{{(0,1)}}(p)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $Binomial({m}, p)$

#### Verossimilhança: $L(p|\mathbf{{X}})=\left(\prod_{{i=1}}^{n}  \binom{{{m}}}{{x_i}} \right) p^{{{n*x}}} (1 - p)^{{{n*(m-x)}}}$

#### Posteriori:

$p|\mathbf{{X}}\sim Beta({n*x + a}, {b+n*(m - x)})$
        '''
      elif verossimilhancas=="Geométrica":
        return fr'''
### Fórmulas matemáticas:

#### Priori:

$p \sim Beta({a},{b})$

$f(p)=\frac{{\Gamma({a+b}) p^{a-1}(1-p)^{b-1}}}{{\Gamma({a})\Gamma({b})}}\mathbb{{I}}_{{(0,1)}}(p)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $\text{{Geométrica}}(p)$

#### Verossimilhança: $L(p|\mathbf{{X}})=p^{n}(1-p)^{{{n*(x-1)}}}$

#### Posteriori:

$p|\mathbf{{X}} \sim Beta({a + n}, {b+n*(x - 1)})$
'''
      elif verossimilhancas=="Binomial negativa":
        return fr'''
### Fórmulas matemáticas:

#### Priori:

$p \sim Beta({a},{b})$

$f(p)=\frac{{\Gamma({a+b}) p^{a-1}(1-p)^{b-1}}}{{\Gamma({a})\Gamma({b})}}\mathbb{{I}}_{{(0,1)}}(p)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $Binomial Negativa({m}, p)$

#### Verossimilhança: $L(p|\mathbf{{X}})=\left( \prod_{{i=1}}^{n}  \binom{{x_i-1}}{{{m-1}}} \right) p^{{{n*m}}}(1-p)^{{{n*(x-m)}}}$

#### Posteriori:

$p|\mathbf{{X}} \sim Beta({a + n*m}, {b+n*(x - m)})$
'''
      elif verossimilhancas=="Exponencial":
        return fr'''
### Fórmulas matemáticas:

#### Priori:

$\lambda \sim Gama({a},{b})$

$f(\lambda)=\frac{{{b**a} \lambda^{{{a - 1}}} e^{{- {b} \lambda}}}}{{\Gamma({a})}} \mathbb{{I}}_{{(0, \infty)}}(\lambda)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $Exponencial(\lambda)$

#### Verossimilhança: $L(\lambda|\mathbf{{X}})=\lambda^{n}e^{{- {n*x} \lambda }}$

#### Posteriori:

$\lambda|\mathbf{{X}}\sim Gama({a+n}, {b+n*x})$
'''
      elif verossimilhancas=="Poisson":
        return fr'''
### Fórmulas matemáticas:

#### Priori:

$\lambda \sim Gama({a},{b})$

$f(\lambda)=\frac{{{b**a} \lambda^{{{a - 1}}} e^{{- {b} \lambda}}}}{{\Gamma({a})}} \mathbb{{I}}_{{(0, \infty)}}(\lambda)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $Poisson(\lambda)$

#### Verossimilhança: $L(\lambda|\mathbf{{X}})= \frac{{e^{{-\lambda}}\lambda^{{{n*x}}}}}{{\prod_{{i=1}}^{n} x_i!}}$

#### Posteriori:

$\lambda|\mathbf{{X}} \sim Gama({n*x+a}, {b+n})$
'''
      elif verossimilhancas=="Gama (b desconhecido)":
        return fr'''
### Fórmulas matemáticas:

#### Priori:

$b \sim Gama({a},{b})$

$f(b)=\frac{{{b**a} b^{{{a - 1}}} e^{{- {b} b}}}}{{\Gamma({a})}} \mathbb{{I}}_{{(0, \infty)}}(b)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $Gama({conhecido},b)$

#### Verossimilhança: $L(b|\mathbf{{X}})=\frac{{\left( \prod_{{i=1}}^{n} x_i \right)^{{{conhecido-1}}} b^{{{n*conhecido}}}e^{{- {n*x} b}}}}{{\Gamma({conhecido})}}$

#### Posteriori:

$p|\mathbf{{X}}\sim Gama({a + n*conhecido}, {b + n*x})$
'''
      elif verossimilhancas=="Normal (média desconhecida)":
        return fr'''
### Fórmulas matemáticas:

#### Priori:

$\mu \sim Normal({a},{b})$

$f(\mu) = \frac{{1}}{{\sqrt{{2 {b} \pi  }}}} \exp\left( -\frac{{(\mu - {a})^2}}{{2 {b} }} \right) \mathbb{{I}}_{{(-\infty, \infty)}}(\mu)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $Normal \left(\mu,{conhecido} \right)$

#### Verossimilhança: $L(\mu|\mathbf{{X}}) = \left(\frac{{1}}{{\sqrt{{2 {conhecido} \pi }}}}\right)^{n} \exp\left(-\frac{{1}}{{2 {conhecido} }}\left({n} \mu^2- {2*n*x} \mu  + \sum_{{i=1}}^{n} x_i^2\right)\right)$

#### Posteriori:

$\mu|\mathbf{{X}}\sim Normal\left( {(n*b*x + conhecido*a)/(n*b + conhecido):.3f} \,,\, {(conhecido*b)/(n*b + conhecido):.3f}\right)$
'''
      else:
        return fr'''
### Fórmulas matemáticas:

#### Priori:

$(\mu,\tau) \sim Normal-Gama({a},{b},{c},{d})$

$f(\mu,\tau)=\frac{{{d**c} \sqrt{{{b}}}\tau^{{{c-0.5}}}e^{{- {d} \tau}}}} {{\Gamma{{({c})}}\sqrt{{2\pi}}}} exp\left( \frac{{{b} \tau \left( \mu-{a} \right)^2}}{{2}} \right)$

#### Modelo estatístico:

$X_1 ... X_n$ condicionalmente independentes e identicamente distribuídos $Normal(\mu,\tau^{{-1}})$

#### Verossimilhança: $L(\mu,\tau|\mathbf{{X}})=\frac{{1}}{{(\sqrt{{2\pi}})^{n}}}\tau^{{{n/2}}}exp(\frac{{- {n*conhecido} \tau}}{{2}})exp \left( \frac{{- {n} \tau \left( \mu-{x} \right)^2}}{{2}} \right)$

#### Posteriori:

$(\mu,\tau)|\mathbf{{X}} \sim Normal-Gama\left( {(b*a+n*x)/(b+n):.3f}, {b+n}, {c+n/2},{(d +0.5*(n*conhecido + (b*n*(x-a)**2)/(b+n))):.3f} \right)$

'''




if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>